# Data preperation 
Read excel files downloaded from Danmarks Statistik 

In [1]:
import pandas as pd

## Mortality 

In [2]:
path = "data/mortality.xlsx"

# Read all sheet names to check structure
xls = pd.ExcelFile(path)

In [3]:
df_before = (
    pd.read_excel(xls, sheet_name=xls.sheet_names[0], header=2, usecols="C:E")
      .rename(columns={'Unnamed: 2': 'AGE', 'Mænd': 'MALE', 'Kvinder': 'FEMALE'})
      .assign(AGE=lambda d: d['AGE'].str.replace(' år', '', regex=False))
      .dropna(subset=['AGE'])
      .assign(AGE=lambda d: d['AGE'].astype(int))
)

# Collapse ≥100 into one row (fixed 100000), then divide columns by 100000
df = (
    df_before
    .pipe(lambda d: pd.concat([
        d.loc[d['AGE'] < 100],
        pd.DataFrame({'AGE': [100], 'MALE': [100000], 'FEMALE': [100000]})
    ], ignore_index=True))
    .assign(
        MALE=lambda d: d['MALE'] / 100000,
        FEMALE=lambda d: d['FEMALE'] / 100000
    )
)
print(df_before.tail(15))
print(df.tail())


     AGE     MALE   FEMALE
96    96  29990.9  23729.0
97    97  32923.5  26303.0
98    98  35995.4  29049.9
99    99  39186.5  31959.3
100  100  42472.7  35016.3
101  101  45826.9  38201.4
102  102  49219.4  41491.3
103  103  52619.0  44858.9
104  104  55994.5  48274.3
105  105  59315.5  51705.9
106  106  62553.5  55121.5
107  107  65682.9  58489.4
108  108  68681.6  61779.9
109  109  71531.9  64965.7
110  110      0.0      0.0
     AGE      MALE    FEMALE
96    96  0.299909  0.237290
97    97  0.329235  0.263030
98    98  0.359954  0.290499
99    99  0.391865  0.319593
100  100  1.000000  1.000000


In [4]:
import re

def interleaved_sort_key(s):
    """
    Sort columns like:
    MORTALITY__FEMALE__AGE_0, MORTALITY__MALE__AGE_0, MORTALITY__FEMALE__AGE_1, ...
    """
    sex = 0 if "__FEMALE__" in s else 1  # FEMALE first
    age = int(re.search(r'AGE_(\d+)', s).group(1))
    return (age, sex)

df_mortality = (
    df
    .melt(id_vars='AGE', var_name='SEX', value_name='VALUE')
    .assign(
        varname=lambda d: 'MORTALITY__' + d['SEX'] + '__AGE_' + d['AGE'].astype(str)
    )
    .pivot_table(values='VALUE', columns='varname')
    .assign(YEAR=2024)
    .set_index('YEAR')
)

# Apply interleaved natural sort
df_mortality = df_mortality[sorted(df_mortality.columns, key=interleaved_sort_key)]

df_mortality.T.head(10)


YEAR,2024
varname,
MORTALITY__FEMALE__AGE_0,0.002093
MORTALITY__MALE__AGE_0,0.002535
MORTALITY__FEMALE__AGE_1,0.000238
MORTALITY__MALE__AGE_1,0.000253
MORTALITY__FEMALE__AGE_2,0.000065
MORTALITY__MALE__AGE_2,0.000123
MORTALITY__FEMALE__AGE_3,0.000046
MORTALITY__MALE__AGE_3,0.000067
MORTALITY__FEMALE__AGE_4,0.000043


In [5]:
path = "data/population.xlsx"

# Read all sheet names to check structure
xls = pd.ExcelFile(path)

In [6]:
df_before = (
    pd.read_excel(xls, sheet_name=xls.sheet_names[0], header=2, usecols="B:D")
      .rename(columns={'Unnamed: 1': 'AGE', 'Mænd': 'MALE', 'Kvinder': 'FEMALE'})
      .assign(AGE=lambda d: d['AGE'].str.replace(' år', '', regex=False))
      .dropna(subset=['AGE'])
 #     .assign(AGE=lambda d: d['AGE'].astype(int))
)

In [7]:
path = "data/population.xlsx"
xls = pd.ExcelFile(path)

df_before = (
    pd.read_excel(xls, sheet_name=xls.sheet_names[0], header=2, usecols="B:D")
      .rename(columns={'Unnamed: 1': 'AGE', 'Mænd': 'MALE', 'Kvinder': 'FEMALE'})
      .assign(AGE=lambda d: d['AGE'].str.replace(' år', '', regex=False))
      .dropna(subset=['AGE'])
)

# Clean AGE column and collapse all rows with AGE >= 100
df = (
    df_before
    .assign(
        AGE=lambda d: d['AGE'].str.extract(r'(\d+)').astype(int)  # extract numeric part (handles "105 og derover")
    )
    .pipe(lambda d: pd.concat([
        d.loc[d['AGE'] < 100],
        pd.DataFrame({
            'AGE': [100],
            'MALE': [d.loc[d['AGE'] >= 100, 'MALE'].sum()],
            'FEMALE': [d.loc[d['AGE'] >= 100, 'FEMALE'].sum()]
        })
    ], ignore_index=True))
)

print(df_before.tail(10))
print(df.tail(10))


                AGE  MALE  FEMALE
96               96  4001    7926
97               97  2877    6534
98               98  1870    4843
99               99  1088    3421
100             100   623    2369
101             101   343    1640
102             102   177    1114
103             103    85     714
104             104    32     389
105  105 og derover    17     357
     AGE   MALE  FEMALE
91    91  12293   16311
92    92  10248   14371
93    93   8751   12931
94    94   7388   12006
95    95   5531    9831
96    96   4001    7926
97    97   2877    6534
98    98   1870    4843
99    99   1088    3421
100  100   1277    6583


In [8]:

df_population = (
    df
    .melt(id_vars='AGE', var_name='SEX', value_name='VALUE')
    .assign(
        varname=lambda d: 'POP__' + d['SEX'] + '__AGE_' + d['AGE'].astype(str)
    )
    .pivot_table(values='VALUE', columns='varname')
    .assign(YEAR=2024)
    .set_index('YEAR')
)

# Interleaved sort by age then sex
df_population = df_population[sorted(df_population.columns, key=interleaved_sort_key)]

df_population.T.tail(14) 


YEAR,2024
varname,
POP__FEMALE__AGE_94,12006.0
POP__MALE__AGE_94,7388.0
POP__FEMALE__AGE_95,9831.0
POP__MALE__AGE_95,5531.0
POP__FEMALE__AGE_96,7926.0
POP__MALE__AGE_96,4001.0
POP__FEMALE__AGE_97,6534.0
POP__MALE__AGE_97,2877.0
POP__FEMALE__AGE_98,4843.0


## Fertility 

In [9]:
path = "data/fertility.xlsx"

# Read all sheet names to check structure
xls = pd.ExcelFile(path)

df_before = (
    pd.read_excel(xls, sheet_name=xls.sheet_names[0], header=2, usecols="B:C")
      .rename(columns={'Unnamed: 1': 'AGE'})
      .assign(AGE=lambda d: d['AGE'].str.replace(' år', '', regex=False))
      .dropna(subset=['AGE'])
)
df_fertility = (df_before
.melt(id_vars='AGE', value_name='VALUE')
.assign(
    varname=lambda d: 'FERTILITY__FEMALE' +  '__AGE_' + d['AGE'].astype(str))
.assign(VALUE = lambda d: d['VALUE']/1000)
    .pivot_table(values='VALUE', columns='varname')
    .assign(YEAR=2024)
    .set_index('YEAR')

       
       
       )

df_fertility

varname,FERTILITY__FEMALE__AGE_15,FERTILITY__FEMALE__AGE_16,FERTILITY__FEMALE__AGE_17,FERTILITY__FEMALE__AGE_18,FERTILITY__FEMALE__AGE_19,FERTILITY__FEMALE__AGE_20,FERTILITY__FEMALE__AGE_21,FERTILITY__FEMALE__AGE_22,FERTILITY__FEMALE__AGE_23,FERTILITY__FEMALE__AGE_24,...,FERTILITY__FEMALE__AGE_40,FERTILITY__FEMALE__AGE_41,FERTILITY__FEMALE__AGE_42,FERTILITY__FEMALE__AGE_43,FERTILITY__FEMALE__AGE_44,FERTILITY__FEMALE__AGE_45,FERTILITY__FEMALE__AGE_46,FERTILITY__FEMALE__AGE_47,FERTILITY__FEMALE__AGE_48,FERTILITY__FEMALE__AGE_49
YEAR,,,,,,,,,,,,,,,,,,,,,
2024,0.0,0.0001,0.0004,0.0009,0.0019,0.0051,0.0078,0.0117,0.0206,0.0316,...,0.0301,0.0214,0.0117,0.0081,0.0048,0.0024,0.0018,0.0008,0.0003,0.0003


In [10]:
df_mortality

varname,MORTALITY__FEMALE__AGE_0,MORTALITY__MALE__AGE_0,MORTALITY__FEMALE__AGE_1,MORTALITY__MALE__AGE_1,MORTALITY__FEMALE__AGE_2,MORTALITY__MALE__AGE_2,MORTALITY__FEMALE__AGE_3,MORTALITY__MALE__AGE_3,MORTALITY__FEMALE__AGE_4,MORTALITY__MALE__AGE_4,...,MORTALITY__FEMALE__AGE_96,MORTALITY__MALE__AGE_96,MORTALITY__FEMALE__AGE_97,MORTALITY__MALE__AGE_97,MORTALITY__FEMALE__AGE_98,MORTALITY__MALE__AGE_98,MORTALITY__FEMALE__AGE_99,MORTALITY__MALE__AGE_99,MORTALITY__FEMALE__AGE_100,MORTALITY__MALE__AGE_100
YEAR,,,,,,,,,,,,,,,,,,,,,
2024,0.002093,0.002535,0.000238,0.000253,0.000065,0.000123,0.000046,0.000067,0.000043,0.000061,...,0.23729,0.299909,0.26303,0.329235,0.290499,0.359954,0.319593,0.391865,1.0,1.0


In [11]:
df_population

varname,POP__FEMALE__AGE_0,POP__MALE__AGE_0,POP__FEMALE__AGE_1,POP__MALE__AGE_1,POP__FEMALE__AGE_2,POP__MALE__AGE_2,POP__FEMALE__AGE_3,POP__MALE__AGE_3,POP__FEMALE__AGE_4,POP__MALE__AGE_4,...,POP__FEMALE__AGE_96,POP__MALE__AGE_96,POP__FEMALE__AGE_97,POP__MALE__AGE_97,POP__FEMALE__AGE_98,POP__MALE__AGE_98,POP__FEMALE__AGE_99,POP__MALE__AGE_99,POP__FEMALE__AGE_100,POP__MALE__AGE_100
YEAR,,,,,,,,,,,,,,,,,,,,,
2024,30000.0,31825.0,30173.0,32016.0,30312.0,32175.0,30363.0,32258.0,30397.0,32268.0,...,7926.0,4001.0,6534.0,2877.0,4843.0,1870.0,3421.0,1088.0,6583.0,1277.0


In [12]:
number_of_years = 100
start_year = 2025 
years= list(range(start_year,start_year+number_of_years))
#years

In [13]:
one_year = pd.concat([df_population,df_mortality,df_fertility],axis=1)
exo_df = pd.concat([one_year]*number_of_years, ignore_index=True)
exo_df.index = years
exo_df.index.name = "YEAR"
exo_df.loc[:,'FRAC_BIRTH__FEMALE'] = 0.50

In [14]:
exo_df.to_parquet("data/exo_population.parquet")
